## Every solution, everywhere, all at once
In this notebook, we ignore the dice and simply check the number of solutions to every possible combination imaginable. 

There are $36C7 = 8,347,680$ mask configurations. Let's compute every possible solution to every possible problem.

Some configurations have more than 100,000 solutions and the average configuration has 1,364 valid solutions so this requires us to find 11,387,941,312 solutions... In 48 minutes on a laptop. Not too shabby.

In [14]:
import math
from concurrent.futures import ProcessPoolExecutor
from itertools import combinations, zip_longest

from genius_square import Printer, Side
from genius_square.experimental import GameState, Solver
from tqdm import tqdm

In [37]:
def get_every_possible_mask(r: int = 7) -> list[int]:
    letters = "ABCDEF"
    numbers = (1, 2, 3, 4, 5, 6)
    all_sides = [Side(f"{letter}{number}") for letter in letters for number in numbers]
    return [sum(tup) for tup in combinations(all_sides, r)]


# Calculate the number of valid solutions for all possible dice rolls
def count_mask(mask: int) -> tuple[int, int]:
    solver = Solver()
    state = GameState(mask)
    soln_count = solver.count_solns(state)
    return mask, soln_count


# There are 8,347,680 configuations from 7 dice.
all_masks = get_every_possible_mask()
assert len(all_masks) == math.comb(36, 7)

masks = all_masks
with ProcessPoolExecutor() as executor:
    results = list(tqdm(executor.map(count_mask, masks, chunksize=1024), total=len(masks)))
    results = sorted(results, key=lambda x: (x[1], x[0]))

count_by_mask = dict(results)
print(f"Solution counts for all {len(count_by_mask):,} possible configurations")


100%|██████████| 8347680/8347680 [48:42<00:00, 2856.61it/s]  


mask,soln_count
i64,i32
1850,0
2869,0
2870,0
3382,0
3386,0
…,…
52914536063008,100593
69269232549889,100593
69269232549920,100593


In [914]:
import polars as pl

df = (
    pl.DataFrame({"mask": count_by_mask.keys(), "soln_count": count_by_mask.values()})
    .select("mask", pl.col("soln_count").cast(pl.Int32))
    .sort("soln_count", "mask")
)

df.write_parquet("../data/every_possible_solution.pq")
df

mask,soln_count
i64,i32
1850,0
2869,0
2870,0
3382,0
3386,0
…,…
52914536063008,100593
69269232549889,100593
69269232549920,100593


In [8]:
import polars as pl

df = pl.read_parquet("../data/every_possible_solution.pq")
df.describe()

statistic,mask,soln_count
str,f64,f64
"""count""",8.34768e6,8.34768e6
"""null_count""",0.0,0.0
"""mean""",1.3522e13,1364.204343
"""std""",1.5628e13,2081.973343
"""min""",319.0,0.0
"""25%""",1.0996e12,280.0
"""50%""",6.6144e12,719.0
"""75%""",2.0891e13,1624.0
"""max""",6.9407e13,100593.0


In [24]:
# Let's look at configurations that have only one solution
single_solutions = df.filter(pl.col("soln_count") == 1)
single_solutions

mask,soln_count
i64,i32
2823176,1
3475972,1
19008784,1
19139332,1
20187652,1
…,…
39694625669120,1
43980549136400,1
43984760603658,1


In [25]:
# An insanely hard puzzle to solve.
found_mask = single_solutions["mask"][180]
solver = Solver()
state = GameState(found_mask)
soln_count = solver.count_solns(state)
print(f"Configuration with only {soln_count} solution.")
solver.solve(state)
state

Configuration with only 1 solution.


🟫🟨🟨🟨🟪🟪
🟫🔘🟨⬜️🔘🟪
🔘🟩🟩⬜️🟥🔘
🟦🟩🟩⬜️🟥🟥
🟦🔘⬛️⬜️🔘🟥
🟦🟦🔘🟧🟧🟧

In [28]:
# Let's build a pretty report of single solutions...
def create_report(solve: bool = False) -> None:
    solver = Solver()
    printer = Printer(empty_piece="🫥")

    solns = []
    for i, mask in enumerate(single_solutions["mask"], start=1):
        state = GameState(mask)
        if solve:
            solver.solve(state)
        repr = printer.str_repr(state.board, state.history)
        solns.append(repr)

    output = "\n".join(solns)

    lines = output.strip().split("\n")
    solutions = [lines[i : i + 6] for i in range(0, len(lines), 6)]
    grouped_solutions = [solutions[i : i + 8] for i in range(0, len(solutions), 8)]

    interleaved = []
    for i, group in enumerate(grouped_solutions, start=1):
        interleaved.append(f"{i}")
        interleaved.extend(
            ["    ".join(row) for row in zip_longest(*group, fillvalue=" " * 6)]  # Join rows side-by-side
        )
        if i % 10 == 0:
            interleaved.append("")
            interleaved.append("")
        # if i > 2:
        #     interleaved.append("etc.")
        #     break

    print("INSANELY HARD PUZZLES")
    print("\n".join(interleaved))


create_report(True)

INSANELY HARD PUZZLES
1
🟩🟩⬜️⬜️⬜️⬜️    ⬜️⬜️⬜️⬜️🟩🟩    🟩🟩⬜️⬜️⬜️⬜️    🟩🟩🟨🟨🟨🟧    🟩🟩⬜️⬜️⬜️⬜️    🟧🟩🟩🟨🟨🟨    🟧🟩🟩🟨🟨🟨    🟨⬜️⬜️⬜️⬜️🟧
🟩🟩🟨🟨🟨🟥    🟥🟨🟨🟨🟩🟩    🟩🟩🟨🟧🟧🟧    🟩🟩🟥🟨⬜️🟧    🟩🟩🟥🟨🟪🟪    🟧🟩🟩⬜️🟨🟥    🟧🟩🟩⬜️🟨🟥    🟨🟨🟩🟩🟦🟧
🟦🟦🟦🟨🟥🟥    🟥🟥🟨🟦🟦🟦    🔘🟨🟨🟦🟦🟦    🔘🟪🟥🟥⬜️🟧    🔘🟥🟥🟨🟨🟪    🟧🔘🟪⬜️🟥🟥    🟧🔘🟪⬜️🟥🟥    🟨🔘🟩🟩🟦🟧
🔘🔘🟦🔘🟥🔘    🔘🟥🔘🟦🔘🔘    ⬛️🔘🟨🟦🟥🔘    🟪🟪🔘🟥⬜️🔘    🟦🟥🔘🟨🔘🔘    🔘🟪🟪⬜️🟥🟦    🔘🟪🟪⬜️🟥🟦    🔘⬛️🔘🟥🟦🟦
🟫🟫🔘⬛️🔘🟪    🟪🔘⬛️🔘🟫🟫    🔘🟪🔘🔘🟥🟥    🔘🔘⬛️🔘⬜️🟦    🟦🔘⬛️🔘🟫🟫    ⬛️🔘🔘⬜️🔘🟦    🟫🔘🔘⬜️🔘🟦    🟫🔘🟥🟥🔘🟪
🟧🟧🟧🔘🟪🟪    🟪🟪🔘🟧🟧🟧    🟪🟪🟫🟫🔘🟥    🟫🟫🔘🟦🟦🟦    🟦🟦🔘🟧🟧🟧    🔘🟫🟫🔘🟦🟦    🟫🔘⬛️🔘🟦🟦    🟫🔘🟥🔘🟪🟪
2
🟨⬜️⬜️⬜️⬜️🟧    🟨⬜️⬜️⬜️⬜️🟧    ⬜️⬜️⬜️⬜️🟩🟩    🟩🟩🟧🟧🟧🟨    ⬜️⬜️⬜️⬜️🟩🟩    ⬜️⬜️⬜️⬜️🟩🟩    🟦🟦🟨🟨🟨🟧    🟨🟧🟧🟧🟦🟦
🟨🟨🟩🟩🟦🟧    🟨🟨🟩🟩🟦🟧    🟦🟨🟨🟨🟩🟩    🟩🟩🟦🟥🟨🟨    🟫🟧🟧🟧🟩🟩    🟦🟨🟨🟨🟩🟩    🟦🟥🟥🟨⬜️🟧    🟨🟨🟩🟩⬜️🟦
🟨🔘🟩🟩🟦🟧    🟨🔘🟩🟩🟦🟧    🟦🔘🟨🟧🟧🟧    ⬜️🔘🟦🟥🟥🟨    🟫🔘🟥🟨🟨🟨    🟦🔘🟨🟧🟧🟧    🟦🔘🟥🟥⬜️🟧    🟨🔘🟩🟩⬜️🟦
🔘⬛️🔘🔘🟦🟦    🔘🟥🔘🔘🟦🟦    🟦🟦🔘🟥🟥🔘    ⬜️🟦🟦🔘🟥🔘    🔘🟥🟥🔘🟨🔘    🟦🟦🔘🔘🟥🔘    🔘⬛️🔘🔘⬜️🔘    🔘⬛️🔘🔘⬜️🔘
🟫🔘🟥🟥🔘🟪    🟥🟥🔘⬛️🔘🟪    🟪🔘🟥🟥🔘🟫    ⬜️🔘🔘⬛️🔘🟪    🟦🟥🔘⬛️🔘🟪    🟪🔘⬛️🔘🟥🟥    🟫🔘🟩🟩⬜️🟪    🟪🔘🟥🟥⬜️🟫
🟫🟥🟥🔘🟪🟪    🟥🟫🟫🔘🟪🟪    🟪🟪🔘⬛️🔘🟫    ⬜️🟫🟫🔘🟪🟪    🟦🟦🟦🔘🟪🟪 